In [1]:
import polars as pl
import numpy as np
import pandas as pd

In [2]:
df = pl.scan_parquet("data/PISA_00-18.parquet")

In [3]:
df.columns

['year',
 'country',
 'school_id',
 'student_id',
 'mother_educ',
 'father_educ',
 'gender',
 'computer',
 'internet',
 'math',
 'read',
 'science',
 'stu_wgt',
 'desk',
 'room',
 'dishwasher',
 'television',
 'computer_n',
 'car',
 'book',
 'wealth',
 'escs',
 '__index_level_0__']

In [4]:
q2 = df.lazy().group_by(pl.col("year")).agg(pl.quantile("math", 0.99).alias("math99"), pl.quantile("read", 0.99).alias("read99"), pl.quantile("science", 0.99).alias("science99")).sort("year")
# this computation is flawed - should use weighted quantile

In [5]:
q = df.lazy().join(q2, on="year").group_by(pl.col("year"), pl.col("country")).agg(pl.mean("math"), pl.mean("read"), pl.mean("science"), (((pl.col("math") > pl.col("math99")) * 100 * pl.col("stu_wgt")).sum() / pl.col("stu_wgt").sum()).alias("in_math99"))

In [6]:
worldbank = pl.scan_parquet("data/worldbank_indicators.parquet").with_columns(pl.col("Time").cast(pl.Int64)).rename({"Time": "year", "Country Code": "country"})

In [7]:
arwu = pl.scan_parquet("data/arwu_2003-2023.parquet").group_by(["country", "year"]).agg(pl.col("rank").count().alias("arwu_ranked_num"), pl.col("score").sum().alias("arwu_sum_score"))

In [8]:
merged = worldbank.join(arwu, left_on=["country", "year"], right_on=["country", "year"], how="left").filter(pl.col("year") >= 2003)

In [9]:
merged = merged.join(q, on=["country", "year"], how="left").with_columns(pl.col("arwu_ranked_num").fill_null(0), pl.col("arwu_sum_score").fill_null(0))

In [10]:
imo_df = pl.scan_parquet("./data/imo_00_23.parquet").select(["code", "total", "year"]).rename({"code": "country", "total": "imo_total_score"})

In [11]:
merged = merged.join(imo_df, on=["country", "year"], how="left").with_columns(pl.col("imo_total_score").fill_null(0))

In [12]:
df2 = merged.collect()

In [13]:
df2.filter(pl.col("country") == "USA").sort("year")

country,year,gdp_pc,gdp_pc_growth,primary_completion,lower_sec_completion,upper_sec_completion,population,arwu_ranked_num,arwu_sum_score,math,read,science,in_math99,imo_total_score
str,i64,f64,f64,f64,f64,f64,u64,u32,f64,f64,f64,f64,f64,f64
"""USA""",2003,39490.274956,1.91648,null,null,null,290107933,161,2446.0,481.961563,493.767805,490.007714,0.565479,188.0
"""USA""",2004,41724.631629,2.895848,98.556938,94.039429,85.977257,292805298,170,2183.7,null,null,null,null,212.0
"""USA""",2005,44123.407068,2.533784,98.422958,93.695877,85.19178,295516599,168,2180.2,null,null,null,null,213.0
"""USA""",2006,46302.00088,1.796486,98.54126,94.191933,85.751007,298379912,167,2190.4,475.177462,null,488.291876,0.486487,154.0
"""USA""",2007,48050.223777,1.04493,null,null,null,301231207,166,2189.4,null,null,null,null,155.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""USA""",2018,62823.309438,2.404868,99.031532,96.029373,89.805359,326838199,217,1899.4,473.142717,500.15018,497.277201,0.427504,212.0
"""USA""",2019,65120.394663,1.829668,null,null,null,328329953,206,1872.3,null,null,null,null,227.0
"""USA""",2020,63528.634303,-3.700953,99.055779,96.46077,90.940521,331511512,206,1745.8,null,null,null,null,183.0


In [14]:
df2.filter(pl.col("country").str.contains("CAN")).sort("year")

country,year,gdp_pc,gdp_pc_growth,primary_completion,lower_sec_completion,upper_sec_completion,population,arwu_ranked_num,arwu_sum_score,math,read,science,in_math99,imo_total_score
str,i64,f64,f64,f64,f64,f64,u64,u32,f64,f64,f64,f64,f64,f64
"""CAN""",2003,28300.463096,0.892853,null,null,null,31644028,24,140.1,521.631895,516.088253,508.497317,1.497084,119.0
"""CAN""",2004,32143.681408,2.134964,null,null,null,31940655,23,137.3,null,null,null,null,132.0
"""CAN""",2005,36382.507916,2.240255,null,null,null,32243753,23,131.8,null,null,null,null,132.0
"""CAN""",2006,40504.060725,1.606178,null,null,79.403389,32571174,22,132.5,517.446121,512.42873,522.5038,1.636177,123.0
"""CAN""",2007,44659.895141,1.063659,null,null,null,32889025,22,132.7,null,null,null,null,98.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""CAN""",2018,46548.638411,1.301965,null,null,null,37065084,27,130.6,503.450271,509.466476,509.894194,2.039556,156.0
"""CAN""",2019,46374.152752,0.455347,null,null,null,37601230,28,131.8,null,null,null,null,144.0
"""CAN""",2020,43562.435831,-6.052474,null,null,null,38007166,28,130.4,null,null,null,null,161.0


In [15]:
df2.shape

(5320, 15)

In [16]:
df2.head()

country,year,gdp_pc,gdp_pc_growth,primary_completion,lower_sec_completion,upper_sec_completion,population,arwu_ranked_num,arwu_sum_score,math,read,science,in_math99,imo_total_score
str,i64,f64,f64,f64,f64,f64,u64,u32,f64,f64,f64,f64,f64,f64
"""AFG""",2003,199.643228,0.927029,null,null,null,22645130,0,0.0,null,null,null,null,0.0
"""AFG""",2004,221.830531,-2.497255,null,null,null,23553551,0,0.0,null,null,null,null,0.0
"""AFG""",2005,254.115274,7.321874,null,null,null,24411191,0,0.0,null,null,null,null,0.0
"""AFG""",2006,274.015394,1.084988,null,null,null,25442944,0,0.0,null,null,null,null,0.0
"""AFG""",2007,376.318296,11.803383,null,null,null,25903301,0,0.0,null,null,null,null,0.0


In [17]:
import statsmodels.formula.api as smf

In [18]:
pd_df = df2.to_pandas()

In [19]:
df2.describe()

statistic,country,year,gdp_pc,gdp_pc_growth,primary_completion,lower_sec_completion,upper_sec_completion,population,arwu_ranked_num,arwu_sum_score,math,read,science,in_math99,imo_total_score
str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""count""","""5320""",5320.0,5126.0,5075.0,1001.0,1098.0,1156.0,5300.0,5320.0,5320.0,365.0,363.0,365.0,366.0,5320.0
"""null_count""","""0""",0.0,194.0,245.0,4319.0,4222.0,4164.0,20.0,0.0,0.0,4955.0,4957.0,4955.0,4954.0,0.0
"""mean""",null,2012.5,15325.512514,2.033909,78.948723,64.568201,51.154957,2.9105e8,2.376128,13.822218,466.910382,464.379821,470.552348,0.913482,27.271241
"""std""",null,5.766823,23704.041944,5.398296,21.951316,25.902321,24.787105,9.1390e8,12.788019,130.259245,54.683512,49.879818,50.3286,1.257516,49.665971
"""min""","""ABW""",2003.0,114.367007,-55.188681,5.16848,2.03617,0.4951,9668.0,0.0,0.0,315.963154,290.918937,326.428759,0.0,0.0
"""25%""",null,2008.0,1706.414917,0.119254,67.309998,45.509998,31.62993,1.502091e6,0.0,0.0,422.987973,425.307999,427.005823,0.069834,0.0
"""50%""",null,2013.0,5536.837483,2.19519,85.60041,65.199997,49.418018,9.758281e6,0.0,0.0,485.39084,480.551538,486.511394,0.503042,0.0
"""75%""",null,2017.0,18814.145817,4.373434,97.933151,89.218559,74.051102,5.9872579e7,0.0,0.0,507.525776,500.845303,508.944869,1.218343,34.0
"""max""","""ZWE""",2022.0,240862.182448,96.95642,100.0,100.0,97.399788,7.9509e9,217.0,2446.0,568.359669,555.079856,563.748407,9.501632,252.0


In [20]:
df2.filter(pl.col("primary_completion").is_not_null()).select(["country", "year"]).group_by("country").agg(pl.max("year"))

country,year
str,i64
"""TZA""",2018
"""TTO""",2009
"""PYF""",2007
"""CYM""",2015
"""SVK""",2019
…,…
"""MNG""",2020
"""KHM""",2021
"""BTN""",2017


In [36]:
pd_df[["math", "in_math99", "gdp_pc", "gdp_pc_growth", "arwu_ranked_num", "primary_completion", "imo_total_score", "population"]].corr()

,math,in_math99,gdp_pc,gdp_pc_growth,arwu_ranked_num,primary_completion,imo_total_score,population
math,1.000000,0.684663,0.525478,-0.051140,0.183521,0.610130,0.176946,-0.144624
in_math99,0.684663,1.000000,0.392608,-0.041073,0.054207,0.162996,0.204582,-0.116232
gdp_pc,0.525478,0.392608,1.000000,-0.065300,0.199751,0.478649,0.107959,-0.106342
gdp_pc_growth,-0.051140,-0.041073,-0.065300,1.000000,-0.013731,-0.079406,0.078716,0.075283
arwu_ranked_num,0.183521,0.054207,0.199751,-0.013731,1.000000,0.210340,0.362395,-0.007854
primary_completion,0.610130,0.162996,0.478649,-0.079406,0.210340,1.000000,0.392065,-0.077606
imo_total_score,0.176946,0.204582,0.107959,0.078716,0.362395,0.392065,1.000000,-0.113109
population,-0.144624,-0.116232,-0.106342,0.075283,-0.007854,-0.077606,-0.113109,1.000000


In [22]:
import plotly.express as px

In [23]:
pd_df["arwu_ranked_num_pc"] = pd_df["arwu_ranked_num"] / pd_df["population"] * 1_000_000

In [26]:
smf.ols("gdp_pc_growth ~ gdp_pc + imo_total_score + math + in_math99 + arwu_ranked_num_pc + primary_completion + lower_sec_completion + upper_sec_completion + population + C(year) ", pd_df[pd_df["year"].isin([2003, 2006, 2009, 2012, 2015, 2018])]).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          gdp_pc_growth   R-squared:                       0.515
Model:                            OLS   Adj. R-squared:                  0.465
Method:                 Least Squares   F-statistic:                     10.38
Date:                Fri, 29 Mar 2024   Prob (F-statistic):           1.19e-15
Time:                        15:45:14   Log-Likelihood:                -362.16
No. Observations:                 152   AIC:                             754.3
Df Residuals:                     137   BIC:                             799.7
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept                5.9349      3.423      1.734      0.085      -0.833      12.703
C(year)[T.2006]          4.5215      1.536      2.943      0.004       1.483       7.560
C(year)[T.2009]         -3.4186      1.500     -2.279      0.024      -6.385      -0.452
C(year)[T.2012]          0.9356      1.537      0.609      0.544      -2.103       3.975
C(year)[T.2015]          1.9884      1.514      1.314      0.191      -1.005       4.982
C(year)[T.2018]          2.0184      1.543      1.308      0.193      -1.033       5.070
gdp_pc               -1.369e-05   1.88e-05     -0.728      0.468   -5.09e-05    2.35e-05
imo_total_score          0.0050      0.005      0.954      0.342      -0.005       0.015
math                    -0.0133      0.009     -1.556      0.122      -0.030       0.004
in_math99                0.3241      0.229      1.418      0.158      -0.128       0.776
arwu_ranked_num_pc      -1.0270      1.142     -0.899      0.370      -3.286       1.232
primary_completion      -0.0049      0.049     -0.100      0.921      -0.102       0.092
lower_sec_completion    -0.0050      0.038     -0.130      0.897      -0.080       0.071
upper_sec_completion     0.0239      0.031      0.781      0.436      -0.037       0.084
population            -4.78e-09   3.73e-09     -1.281      0.202   -1.22e-08     2.6e-09
==============================================================================
Omnibus:                       23.382   Durbin-Watson:                   2.188
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               96.863
Skew:                          -0.373   Prob(JB):                     9.25e-22
Kurtosis:                       6.839   Cond. No.                     1.55e+09
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.55e+09. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [33]:
smf.ols("gdp_pc_growth ~ gdp_pc + imo_total_score*gdp_pc + arwu_ranked_num_pc*gdp_pc + primary_completion + lower_sec_completion + upper_sec_completion + np.log(population) + C(year) ", pd_df).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          gdp_pc_growth   R-squared:                       0.386
Model:                            OLS   Adj. R-squared:                  0.367
Method:                 Least Squares   F-statistic:                     21.13
Date:                Fri, 29 Mar 2024   Prob (F-statistic):           9.78e-81
Time:                        15:47:34   Log-Likelihood:                -2495.5
No. Observations:                 971   AIC:                             5049.
Df Residuals:                     942   BIC:                             5190.
Df Model:                          28                                         
Covariance Type:            nonrobust                                         
=============================================================================================
                                coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------
Intercept                     4.4273      1.569      2.821      0.005       1.348       7.507
C(year)[T.2004]               4.2641      0.949      4.495      0.000       2.402       6.126
C(year)[T.2005]               2.7470      0.931      2.950      0.003       0.920       4.574
C(year)[T.2006]               3.8372      0.905      4.239      0.000       2.061       5.613
C(year)[T.2007]               3.9450      0.889      4.438      0.000       2.200       5.690
C(year)[T.2008]               1.9989      0.886      2.257      0.024       0.261       3.737
C(year)[T.2009]              -2.9379      0.876     -3.355      0.001      -4.656      -1.219
C(year)[T.2010]               2.3078      0.866      2.666      0.008       0.609       4.007
C(year)[T.2011]               2.7371      0.866      3.161      0.002       1.038       4.436
C(year)[T.2012]               1.2027      0.874      1.376      0.169      -0.513       2.918
C(year)[T.2013]               1.2915      0.884      1.462      0.144      -0.443       3.026
C(year)[T.2014]               1.4516      0.873      1.663      0.097      -0.261       3.164
C(year)[T.2015]               1.2507      0.855      1.462      0.144      -0.428       2.929
C(year)[T.2016]               1.3128      0.867      1.514      0.130      -0.389       3.015
C(year)[T.2017]               1.5735      0.871      1.806      0.071      -0.136       3.283
C(year)[T.2018]               1.4552      0.864      1.684      0.093      -0.241       3.151
C(year)[T.2019]               1.4315      0.867      1.652      0.099      -0.269       3.132
C(year)[T.2020]              -6.9378      0.900     -7.705      0.000      -8.705      -5.171
C(year)[T.2021]               3.1077      1.049      2.962      0.003       1.049       5.167
C(year)[T.2022]               2.6153      1.374      1.904      0.057      -0.081       5.312
gdp_pc                    -4.307e-05   9.31e-06     -4.627      0.000   -6.13e-05   -2.48e-05
imo_total_score               0.0113      0.003      3.485      0.001       0.005       0.018
imo_total_score:gdp_pc    -7.339e-08   9.55e-08     -0.768      0.442   -2.61e-07    1.14e-07
arwu_ranked_num_pc           -2.2709      1.050     -2.164      0.031      -4.331      -0.211
arwu_ranked_num_pc:gdp_pc  4.504e-05   2.07e-05      2.180      0.029     4.5e-06    8.56e-05
primary_completion           -0.0052      0.011     -0.469      0.639      -0.027       0.016
lower_sec_completion         -0.0008      0.016     -0.053      0.957      -0.032       0.030
upper_sec_completion          0.0091      0.013      0.718      0.473      -0.016       0.034
np.log(population)           -0.2110      0.074     -2.863      0.004      -0.356      -0.066
===========================================================================

In [62]:
fig = px.scatter(df2, y="in_math99", x="gdp_pc_growth", color='gdp_pc', hover_data=["country", "year"])
fig.show()

In [63]:
fig = px.scatter(pd_df, y="arwu_ranked_num_pc", x="gdp_pc_growth", color='gdp_pc', hover_data=["country", "year"])
fig.show()

In [84]:
fig = px.scatter(pd_df, y="math", x="gdp_pc_growth", color='gdp_pc', hover_data=["country", "year"])
fig.show()

In [91]:
fig = px.scatter(df2.filter(pl.col("country").str.contains("HKG")), y="primary_completion", x="year", color='gdp_pc', hover_data="country")
fig.show()

In [92]:
from statsmodels.tsa.api import VAR
usa_df = pd_df[pd_df["country"].str.contains("USA")]

In [100]:
data = usa_df[["gdp_pc_growth", "gdp_pc", "arwu_ranked_num"]]
data.index = pd.DatetimeIndex(pd.to_datetime(usa_df["year"], format="%Y"))
data = data.dropna()
data.shape

(20, 3)

In [101]:
model = VAR(data)

/Users/jamesxu/miniforge3/envs/442research/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

No frequency information was provided, so inferred frequency YS-JAN will be used.



In [103]:
model.fit(maxlags=3, ic='aic').summary()

  Summary of Regression Results   
Model:                         VAR
Method:                        OLS
Date:           Fri, 29, Mar, 2024
Time:                     15:15:55
--------------------------------------------------------------------
No. of Equations:         3.00000    BIC:                    20.3150
Nobs:                     17.0000    HQIC:                   18.9908
Log likelihood:          -202.545    FPE:                2.57107e+08
AIC:                      18.8446    Det(Omega_mle):     6.41756e+07
--------------------------------------------------------------------
Results for equation gdp_pc_growth
                        coefficient       std. error           t-stat            prob
-------------------------------------------------------------------------------------
const                     -2.800282         6.040017           -0.464           0.643
L1.gdp_pc_growth           0.145326         1.199712            0.121           0.904
L1.gdp_pc                 -0.001

In [34]:
df2.write_parquet("./data/combined_all.parquet")

In [35]:
df2.write_csv("./data/combined_all.csv")